In [8]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageOps
import os
import matplotlib.pyplot as plt
import os
import time
import signal
import numpy as np
import pickle
import pandas as pd

from sklearn.model_selection import train_test_split

In [10]:
def img_resize(image,desired_width = 244,desired_height = 244):
    current_height, current_width = image.shape
    aspect_ratio = current_width / current_height

    if aspect_ratio > 1:
        # Landscape image
        new_width = desired_width
        new_height = int(new_width / aspect_ratio)
    else:
        # Portrait or square image
        new_height = desired_height
        new_width = int(new_height * aspect_ratio)

    image = cv2.resize(image, (new_width, new_height))

    # Add padding to match desired width and height
    top_pad = (desired_height - new_height) // 2
    bottom_pad = desired_height - new_height - top_pad
    left_pad = (desired_width - new_width) // 2
    right_pad = desired_width - new_width - left_pad

    image = cv2.copyMakeBorder(image, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=0)

    return image

def training_existing_data():
    dataset_folder = "source_pre"
    images=[]
    ratings=[]
    for filename in os.listdir(dataset_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            rating= int(filename.split('_')[1][0])
#             print(rating)

            image = cv2.imread(dataset_folder + '/' + filename)
            # print(image)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            image= img_resize(gray_image)
            if rating >=1 and rating<=5:
                images.append(image)
                ratings.append(rating)
#             cv2.imshow('image', image)
            # time.sleep(10)
#             cv2.waitKey(25)
    images = np.array(images)
    ratings = np.array(ratings)
#     print("Images shape:", images.shape)
#     print("Ratings shape:", ratings.shape)
    return images,ratings

images,ratings=training_existing_data()

In [66]:
X_train, X_val, y_train, y_val = train_test_split(images, ratings, test_size=0.2, random_state=42)

In [67]:
X_train.shape,y_train.shape,X_val.shape,y_val.shape

((534, 244, 244), (534,), (134, 244, 244), (134,))

In [68]:
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

In [69]:
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_val[0].shape

(244, 244, 1)

In [70]:
y_train_categorical = tf.keras.utils.to_categorical(y_train - 1, num_classes=5)
y_val_categorical = tf.keras.utils.to_categorical(y_val - 1, num_classes=5)


In [71]:
# Convert the data type to int32
y_train_categorical = y_train_categorical.astype(np.int32)
y_val_categorical = y_val_categorical.astype(np.int32)
y_train_categorical

array([[0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0]], dtype=int32)

In [72]:
# X_train = X_train.reshape(-1, 244, 244, 1)
# X_val = X_val.reshape(-1, 244, 244, 1)

In [76]:
X_train.shape,y_train_categorical.shape,X_val.shape,y_val_categorical.shape

((534, 244, 244, 1), (534, 5), (134, 244, 244, 1), (134, 5))

# model creation

In [39]:
import tensorflow as tf
from tensorflow.keras import layers


In [81]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(244, 244, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax')  # Use softmax activation for multi-class classification
])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.009)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 242, 242, 32)      320       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 121, 121, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 119, 119, 64)      18496     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 59, 59, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 57, 57, 128)       73856     
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 28, 28, 128)     

In [83]:
history = model.fit(X_train, y_train_categorical, validation_data=(X_val, y_val_categorical), epochs=20, batch_size=32)


Epoch 1/20
17/17 [==============================] - 14s 815ms/step - loss: 1.3964 - accuracy: 0.3109 - val_loss: 1.3869 - val_accuracy: 0.3881
Epoch 2/20
17/17 [==============================] - 14s 817ms/step - loss: 1.3903 - accuracy: 0.3539 - val_loss: 1.3506 - val_accuracy: 0.3134
Epoch 3/20
17/17 [==============================] - 14s 806ms/step - loss: 1.3834 - accuracy: 0.3352 - val_loss: 1.3636 - val_accuracy: 0.3881
Epoch 4/20
17/17 [==============================] - 14s 795ms/step - loss: 1.3828 - accuracy: 0.3221 - val_loss: 1.3601 - val_accuracy: 0.3881
Epoch 5/20
17/17 [==============================] - 14s 816ms/step - loss: 1.3790 - accuracy: 0.3539 - val_loss: 1.3549 - val_accuracy: 0.3881
Epoch 6/20
17/17 [==============================] - 14s 826ms/step - loss: 1.3871 - accuracy: 0.3539 - val_loss: 1.3556 - val_accuracy: 0.3881
Epoch 7/20
17/17 [==============================] - 14s 801ms/step - loss: 1.3794 - accuracy: 0.3539 - val_loss: 1.3931 - val_accuracy: 0.3881